In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import words
from nltk.stem import LancasterStemmer
import os
from pathlib import Path

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
stop_words=stopwords.words("english")
lemmatizer = WordNetLemmatizer()
dic = words.words()
ls = LancasterStemmer()

In [ ]:
os.chdir('/content/drive/MyDrive/Research/Single_Documents/2. Data_cleaning/')
root_dir = os.getcwd()
root_dir

In [ ]:
dataset  = pd.read_csv('/content/drive/MyDrive/Research/duc_2001.csv')
dataset = dataset.iloc[:,1:]

In [ ]:
dataset = dataset.loc[:,['Docno', 'Dir', 'Text', 'Chars', 'Words']]
# dataset = dataset.head(15)

In [ ]:
def remove_punctuation(text):
  for punctuation in string.punctuation:
    text = text.replace(punctuation, '')
  return text

In [ ]:
def remove_stopword(text):
  return " ".join([i for i in word_tokenize(text) if i not in stop_words])

In [ ]:
def lemmatization(s):
  return " ".join([lemmatizer.lemmatize(word) for word in s.split()])

In [ ]:
def stemming(text):
  return " ".join([ls.stem(i) if ls.stem(i) in dic else i for i in text.split()])

In [ ]:
def get_clean_text(text):
  sentence = sent_tokenize(text)
  sentence_dataset = pd.DataFrame(sentence, columns = ['sentence'])
  sentence_dataset['Removed_punctuation'] = sentence_dataset['sentence'].apply(remove_punctuation)
  sentence_dataset['Removed_Stopword'] = sentence_dataset['Removed_punctuation'].apply(remove_stopword)
  # sentence_dataset['Lemmatized_text'] = sentence_dataset['Removed_Stopword'].apply(lemmatization)
  # sentence_dataset['Stemmed_text'] = sentence_dataset['Lemmatized_text'].apply(stemming)
  sentence_dataset['Stemmed_text'] = sentence_dataset['Removed_punctuation'].apply(stemming)
  sentence_dataset['Words'] = sentence_dataset['Stemmed_text'].str.split().str.len()
  sentence_dataset['Chars'] = sentence_dataset['Stemmed_text'].str.len()
  return sentence_dataset

In [ ]:
dataset.loc[dataset['Text'].isna()]

Docno   Dir Text  Chars  Words
288  WSJ910628-0109  d45h  NaN      0      0

In [ ]:
dataset = dataset.dropna()

In [ ]:
dataset = dataset.drop_duplicates(subset = ['Docno', 'Dir'], keep = 'first')

In [ ]:
dataset.shape

(307, 5)

In [ ]:
dataset['sentence_data'] = dataset['Text'].apply(get_clean_text)

In [ ]:
def generate_files(dir):
  dataset_final = pd.DataFrame()
  for v in set(dir):
      root_location = Path("/content/drive/MyDrive/Research/Single_Documents/2. Data_cleaning/" + v)
      os.makedirs(root_location, exist_ok=True)
      temp_df = dataset[dataset["Dir"] == v]
      for index, row in temp_df.iterrows():
          with open(root_location / (str(row["Docno"]) + ".txt"), "w") as f:
              f.write(" ".join(list(row['sentence_data']['Stemmed_text'])))
          row['sentence_data']['Docno'] = row['Docno']
          row['sentence_data']['Dir'] = v
          dataset_final = dataset_final.append(row['sentence_data'])
  dataset_final = dataset_final[['Docno','Dir', 'sentence', 'Removed_punctuation', 'Removed_Stopword', 'Lemmatized_text', 'Stemmed_text', 'Words', 'Chars']]
  dataset_final.to_csv('/content/drive/MyDrive/Research/Single_Documents/2. Data_cleaning/duc_2001_clean.csv')
generate_files(dataset['Dir'])

In [ ]:
stpw=stopwords.words('english')

In [ ]:
print(stpw)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
print(len(stpw))

179
